---
title: "A Replication of Karlan and List (2007)"
author: "Jenny Shyu"
date: today
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---





## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a field experiment to test the effectiveness of different fundraising letters. They sent out 50,000 fundraising letters to potential donors, randomly assigning each letter to one of three treatments: a standard letter, a matching grant letter, or a challenge grant letter. They published the results of this experiment in the _American Economic Review_ in 2007. The article and supporting data are available from the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action as part of [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).

This project seeks to replicate their results. The goal of the experiment was to test whether offering a matching donation—in which a lead donor promises to match contributions from other individuals—would increase the likelihood and/or size of charitable donations. In addition to testing whether matching grants were effective in general, Karlan and List also explored whether the size of the match mattered. Some participants were told that every dollar they donated would be matched 1:1, while others were offered more generous matches (2:1 or 3:1), allowing the researchers to test for differences in donor behavior across match sizes.

The experiment is notable for its scale, randomization, and use of real-world donor behavior, which together provide credible evidence of causal effects. Because the fundraising letters were identical in every respect except for the treatment condition, any differences in outcomes across groups can be attributed to the match offer itself. This approach allows for insights not only into how people respond to incentives, but also into broader questions about social influence, perceived impact, and behavioral nudges in charitable giving.


## Data

### Description


In [ ]:
import pandas as pd

data = pd.read_stata("karlan_list_2007.dta")
data.head()

In [ ]:
data.describe()

:::: {.callout-note collapse="true"}
### Variable Definitions

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::


### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.

_todo: test a few variables other than the key outcome variables (for example, test months since last donation) to see if the treatment and control groups are statistically significantly different at the 95% confidence level. Do each as a t-test and separately as a linear regression, and confirm you get the exact same results from both methods. When doing a t-test, use the formula in the class slides. When doing the linear regression, regress for example mrm2 on treatment and look at the estimated coefficient on the treatment variable. It might be helpful to compare parts of your analysis to Table 1 in the paper. Be sure to comment on your results (hint: why is Table 1 included in the paper)._

As an ad hoc test of the randomization mechanism, I compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another at the 95% confidence level. If randomization was properly executed, we should expect no statistically significant differences in pre-treatment characteristics between the groups.

I begin by testing the variable mrm2, which captures the number of months since the last donation. This variable is useful for checking balance because it is unrelated to the treatment assignment and reflects donor history.


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from scipy import stats


data = pd.read_stata("karlan_list_2007.dta")

vars_to_test = ["mrm2", "amount", "years", "freq"]
results = []

for var in vars_to_test:
    subset = data[["treatment", var]].dropna()
    control = subset[subset["treatment"] == 0][var]
    treatment = subset[subset["treatment"] == 1][var]
    
    # T-test
    t_stat, p_val = stats.ttest_ind(treatment, control, equal_var=False)
    
    # Linear regression
    regression = smf.ols(f"{var} ~ treatment", data=subset).fit()
    coef = regression.params["treatment"]
    reg_p = regression.pvalues["treatment"]
    
    # Difference in means
    diff = treatment.mean() - control.mean()
    
    results.append({
        "Variable": var,
        "Diff (Treat - Control)": round(diff, 5),
        "T-test p-value": round(p_val, 5),
        "Regression Coef": round(coef, 5),
        "Regression p-value": round(reg_p, 5)
    })

pd.DataFrame(results)


The table of results above shows no statistically significant differences at the 5% level for any variable (p-values > 0.05), though amount is marginally close (p ≈ 0.06 in the regression). This is consistent with proper random assignment.

These checks are similar to what Karlan and List report in Table 1 of the original paper, which reassures readers that the treatment effect estimates later in the paper can be interpreted as causal. If pre-treatment covariates are balanced, then observed differences in outcomes are more likely attributable to the randomized treatment itself.

#### Interpretation
These results mirror those presented in Table 1 of Karlan and List (2007), which shows no significant differences between the groups in prior donation behavior and demographic characteristics. Table 1 serves to reassure the reader that any observed treatment effects later in the analysis can be confidently attributed to the randomized intervention rather than pre-existing differences between groups.

## Experimental Results

### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 

_todo: make a barplot with two bars. Each bar is the proportion of people who donated. One bar for treatment and one bar for control._

_todo: run a t-test between the treatment and control groups on the binary outcome of whether any charitable donation was made. Also run a bivariate linear regression that demonstrates the same finding. (It may help to confirm your calculations match Table 2a Panel A.) Report your statistical results and interpret them in the context of the experiment (e.g., if you found a difference with a small p-value or that was statistically significant at some threshold, what have you learned about human behavior? Use mostly English words, not numbers or stats, to explain your finding.)_

_todo: run a probit regression where the outcome variable is whether any charitable donation was made and the explanatory variable is assignment to treatment or control. Confirm that your results replicate Table 3 column 1 in the paper._

#### Donation Rate by Group


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate donation rates
donation_rates = data.groupby("treatment")["gave"].mean().reset_index()
donation_rates["group"] = donation_rates["treatment"].map({0: "Control", 1: "Treatment"})

# Create barplot
plt.figure(figsize=(6, 4))
ax = sns.barplot(data=donation_rates, x="group", y="gave")
plt.ylabel("Proportion Donated")
plt.xlabel("")
plt.title("Donation Rate by Group")
plt.ylim(0, 0.03)
plt.grid(axis='y')

# Add percentage labels on top
for i, val in enumerate(donation_rates["gave"]):
    ax.text(i, val + 0.0005, f"{val:.3%}", ha='center', va='bottom')

plt.tight_layout()
plt.show()

This bar plot shows that the treatment group—who received matching grant letters—had a higher donation rate than the control group.

#### T-Test and Linear Regression


In [ ]:
treat_gave = data[data['treatment'] == 1]['gave']
control_gave = data[data['treatment'] == 0]['gave']
t_stat, t_pval = stats.ttest_ind(treat_gave, control_gave, equal_var=False)

# Format output
pd.DataFrame([{
    "T-test Statistic": round(t_stat, 3),
    "T-test p-value": round(t_pval, 5)
}])

In [ ]:
gave_regression = smf.ols("gave ~ treatment", data=data).fit()
coef = gave_regression.params["treatment"]
std_err = gave_regression.bse["treatment"]
p_val = gave_regression.pvalues["treatment"]
conf_int = gave_regression.conf_int().loc["treatment"]

# Format output
pd.DataFrame([{
    "Treatment Coefficient": round(coef, 5),
    "Standard Error": round(std_err, 5),
    "p-value": round(p_val, 5),
    "95% CI Lower": round(conf_int[0], 5),
    "95% CI Upper": round(conf_int[1], 5)
}])

The difference in donation rates is statistically significant at the 1% level.

The treatment group is more likely to donate, increasing the probability of giving by about 0.42 percentage points.

This replicates the result from Table 2A Panel A in Karlan & List (2007), showing that a match offer significantly boosts participation.

OLS regression shows a statistically significant positive coefficient (≈ 0.0042) on the treatment variable. This confirms the t-test: assignment to the treatment group increased the likelihood of making a donation.

This suggests that even a small behavioral nudge like mentioning a matching donation makes people more likely to contribute to charity. People respond to the perception of increased impact.


In [ ]:
# Probit model
import statsmodels.api as sm

probit_model = smf.probit("gave ~ treatment", data=data).fit()
coef = probit_model.params["treatment"]
std_err = probit_model.bse["treatment"]
p_val = probit_model.pvalues["treatment"]
conf_int = probit_model.conf_int().loc["treatment"]

# Output summary
pd.DataFrame([{
    "Probit Coefficient": round(coef, 5),
    "Standard Error": round(std_err, 5),
    "p-value": round(p_val, 5),
    "95% CI Lower": round(conf_int[0], 5),
    "95% CI Upper": round(conf_int[1], 5)
}])

The probit model replicates Table 3, Column 1 of Karlan and List (2007), with a significant positive treatment effect (coefficient ≈ 0.087, p ≈ 0.002). This again confirms that individuals are more likely to donate when offered a matching grant.

Together, these results demonstrate a consistent and statistically significant treatment effect, providing strong evidence that the framing of charitable solicitations matters for donor behavior.

The match incentive not only has a practical impact but also a statistically robust one, even under a probit framework.

### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

_todo: Use a series of t-tests to test whether the size of the match ratio has an effect on whether people donate or not. For example, does the 2:1 match rate lead increase the likelihood that someone donates as compared to the 1:1 match rate? Do your results support the "figures suggest" comment the authors make on page 8?_

_todo: Assess the same issue using a regression. Specifically, create the variable `ratio1` then regress `gave` on `ratio1`, `ratio2`, and `ratio3` (or alternatively, regress `gave` on the categorical variable `ratio`). Interpret the coefficients and their statistical precision._

_todo: Calculate the response rate difference between the 1:1 and 2:1 match ratios and the 2:1 and 3:1 ratios.  Do this directly from the data, and do it by computing the differences in the fitted coefficients of the previous regression. what do you conclude regarding the effectiveness of different sizes of matched donations?_

also include p-velue 2:1 vs 1:1, 3:1 vs 2:1, 3:1 vs 1:1

for ols regresssion only have intercept, ration2, ratio3

raw difference (2:1-1:1)
raw difference (3:1 - 2:1)
fitted difference (2:1 - 1:1)
fitted difference (3:1 - 2:1)


#### Response Rates by Match Ratio


In [ ]:
matched_data = data[(data["treatment"] == 1) & (data["ratio"].isin([1, 2, 3]))]

# Calculate means
response_rates = matched_data.groupby("ratio")["gave"].mean()

# Separate groups for pairwise comparisons
gave_1 = matched_data[matched_data["ratio"] == 1]["gave"]
gave_2 = matched_data[matched_data["ratio"] == 2]["gave"]
gave_3 = matched_data[matched_data["ratio"] == 3]["gave"]

# T-tests for pairwise comparisons
from scipy import stats

summary = pd.DataFrame({
    "Comparison": ["2:1 vs 1:1", "3:1 vs 2:1", "3:1 vs 1:1"],
    "p-value": [
        round(stats.ttest_ind(gave_2, gave_1, equal_var=False).pvalue, 5),
        round(stats.ttest_ind(gave_3, gave_2, equal_var=False).pvalue, 5),
        round(stats.ttest_ind(gave_3, gave_1, equal_var=False).pvalue, 5)
    ],
    "Rate A": [round(gave_2.mean(), 5), round(gave_3.mean(), 5), round(gave_3.mean(), 5)],
    "Rate B": [round(gave_1.mean(), 5), round(gave_2.mean(), 5), round(gave_1.mean(), 5)],
    "Difference (A - B)": [
        round(gave_2.mean() - gave_1.mean(), 5),
        round(gave_3.mean() - gave_2.mean(), 5),
        round(gave_3.mean() - gave_1.mean(), 5)
    ]
})

summary


Observed donation rates:

1:1 match — 2.07%

2:1 match — 2.26%

3:1 match — 2.27%

The increase from 1:1 to 2:1 and 3:1 appears small.

None of the pairwise comparisons are statistically significant. This supports the paper’s statement on page 8 that larger match ratios do not lead to meaningfully higher donation rates.




#### Regression: Match Ratio Effects


In [ ]:
# Regression with dummy variables (baseline: 1:1 match)
import statsmodels.formula.api as smf

matched_data["ratio2"] = (matched_data["ratio"] == 2).astype(int)
matched_data["ratio3"] = (matched_data["ratio"] == 3).astype(int)

reg_model = smf.ols("gave ~ ratio2 + ratio3", data=matched_data).fit()

# Clean formatted output
coefs = reg_model.params
stderr = reg_model.bse
pvals = reg_model.pvalues
confint = reg_model.conf_int()

pd.DataFrame({
    "Coefficient": coefs.round(5),
    "Std. Error": stderr.round(5),
    "p-value": pvals.round(5),
    "95% CI Lower": confint[0].round(5),
    "95% CI Upper": confint[1].round(5)
}).loc[["Intercept", "ratio2", "ratio3"]].reset_index().rename(columns={"index": "Term"})


Regression results:

The baseline (1:1 match) donation rate is about 2.07%.

The 2:1 match effect: +0.19 percentage points (not statistically significant).

The 3:1 match effect: +0.20 percentage points (also not statistically significant).

Neither the 2:1 nor 3:1 match ratio coefficients are statistically significant. The results suggest that changing the size of the match ratio does not significantly change donation likelihood relative to 1:1.


#### Difference in Response Rates (Data vs. Regression Coefficients)


In [ ]:
# Mean differences and coefficient comparisons
pd.DataFrame([
    {
        "Comparison": "2:1 vs 1:1",
        "Raw Mean Difference": round(gave_2.mean() - gave_1.mean(), 5),
        "Regression Coefficient": round(reg_model.params["ratio2"], 5)
    },
    {
        "Comparison": "3:1 vs 2:1",
        "Raw Mean Difference": round(gave_3.mean() - gave_2.mean(), 5),
        "Regression Coefficient Diff (3 - 2)": round(reg_model.params["ratio3"] - reg_model.params["ratio2"], 5)
    },
    {
        "Comparison": "3:1 vs 1:1",
        "Raw Mean Difference": round(gave_3.mean() - gave_1.mean(), 5),
        "Regression Coefficient": round(reg_model.params["ratio3"], 5)
    }
])


The differences in donation rates between 1:1, 2:1, and 3:1 match offers are very small and not statistically significant. These findings replicate the comment on page 8 of Karlan and List (2007): "Larger match ratios relative to a smaller match ratio had no additional impact."

This suggests that donors may respond to the presence of a match, but not necessarily to the size of the match. Psychologically, the idea of having one's donation matched could serve as a signal of trust or endorsement—but the exact multiplier does not further influence behavior.

### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.

_todo: Calculate a t-test or run a bivariate linear regression of the donation amount on the treatment status. What do we learn from doing this analysis?_

_todo: now limit the data to just people who made a donation and repeat the previous analysis. This regression allows you to analyze how much respondents donate conditional on donating some positive amount. Interpret the regression coefficients -- what did we learn? Does the treatment coefficient have a causal interpretation?_ 

_todo: Make two plot: one for the treatment group and one for the control. Each plot should be a histogram of the donation amounts only among people who donated. Add a red vertical bar or some other annotation to indicate the sample average for each plot._

charts side by side and scale
#### All Respondents: Does Treatment Affect Amount Donated?


In [ ]:
amount_reg_all = smf.ols("amount ~ treatment", data=data).fit()

pd.DataFrame([{
    "Treatment Coefficient": round(amount_reg_all.params["treatment"], 5),
    "Std. Error": round(amount_reg_all.bse["treatment"], 5),
    "t-statistic": round(amount_reg_all.tvalues["treatment"], 3),
    "p-value": round(amount_reg_all.pvalues["treatment"], 5),
    "95% CI Lower": round(amount_reg_all.conf_int().loc["treatment", 0], 5),
    "95% CI Upper": round(amount_reg_all.conf_int().loc["treatment", 1], 5)
}])


Coefficient on treatment ≈ 0.15

p-value ≈ 0.063

Among all individuals (including non-donors), the treatment group donated about $0.15 more on average. This effect is marginally significant (p ≈ 0.063). It suggests that matching increases expected donations slightly, but much of that effect may be driven by more people giving (rather than giving more).

This suggests that those who received a matching letter donated slightly more on average. However, the result is only marginally significant (at the 10% level). This weak evidence may indicate that the offer of a match has a small impact on the total amount donated—though for most people, the presence of the match does not substantially alter donation size.

#### Conditional on Donation: Do Donors Give More if Matched?


In [ ]:
# Subset to donors only
donors = data[data["gave"] == 1]

# Linear regression among donors only
amount_reg_donors = smf.ols("amount ~ treatment", data=donors).fit()

pd.DataFrame([{
    "Treatment Coefficient": round(amount_reg_donors.params["treatment"], 5),
    "Std. Error": round(amount_reg_donors.bse["treatment"], 5),
    "t-statistic": round(amount_reg_donors.tvalues["treatment"], 3),
    "p-value": round(amount_reg_donors.pvalues["treatment"], 5),
    "95% CI Lower": round(amount_reg_donors.conf_int().loc["treatment", 0], 5),
    "95% CI Upper": round(amount_reg_donors.conf_int().loc["treatment", 1], 5)
}])


Coefficient on treatment ≈ -1.67

p-value = 0.561

Among those who did donate, receiving a match letter did not significantly change the amount given. In fact, the coefficient is slightly negative, though not significant. Thus, we conclude that while match offers may increase the number of donors, they do not cause donors to give more, conditional on giving.

This coefficient does not have a strong causal interpretation, because donation decisions and donation amounts are jointly determined and the sample is selected on gave == 1.

#### Distribution of Donations Among Donors


In [ ]:
treatment_donors = data[(data["treatment"] == 1) & (data["gave"] == 1)]
control_donors = data[(data["treatment"] == 0) & (data["gave"] == 1)]

mean_treat = treatment_donors["amount"].mean()
mean_control = control_donors["amount"].mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

sns.histplot(treatment_donors["amount"], bins=30, color="orange", edgecolor="black", ax=axes[0])
axes[0].axvline(mean_treat, color='red', linestyle='--', label=f"Mean: ${mean_treat:.2f}")
axes[0].set_title("Treatment Group")
axes[0].set_xlabel("Donation Amount")
axes[0].set_ylabel("Frequency")
axes[0].legend()

sns.histplot(control_donors["amount"], bins=30, color="orange", edgecolor="black", ax=axes[1])
axes[1].axvline(mean_control, color='red', linestyle='--', label=f"Mean: ${mean_control:.2f}")
axes[1].set_title("Control Group")
axes[1].set_xlabel("Donation Amount")
axes[1].legend()

plt.tight_layout()
plt.show()


Two histograms show the distribution of donation amounts for the treatment group and control group, restricted to those who donated. The vertical red line marks the average for each group:

Treatment Mean: ~$43.87

Control Mean: ~$45.54

Both distributions are right-skewed, with most donors giving small amounts and a few contributing large sums. There is no visible shift in the average due to the treatment.

#### Conclusion
These analyses support the idea that matching offers increase response rate, but do not change how much people give once they’ve decided to donate. This distinction is important for fundraising strategies: matching may motivate more people to give, but it doesn't necessarily increase per-donor revenue.



## Simulation Experiment

As a reminder of how the t-statistic "works," in this section I use simulation to demonstrate the Law of Large Numbers and the Central Limit Theorem.

Suppose the true distribution of respondents who do not get a charitable donation match is Bernoulli with probability p=0.018 that a donation is made. 

Further suppose that the true distribution of respondents who do get a charitable donation match of any size  is Bernoulli with probability p=0.022 that a donation is made.


### Law of Large Numbers

_to do:  Make a plot like those on slide 43 from our first class and explain the plot to the reader. To do this, you will simulate 100,00 draws from the control distribution and 10,000 draws from the treatment distribution. You'll then calculate a vector of 10,000 differences, and then you'll plot the cumulative average of that vector of differences. Comment on whether the cumulative average approaches the true difference in means._


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p_control = 0.018
p_treatment = 0.022
n_sim = 10000
np.random.seed(42)
# Simulate 10,000 binary outcomes for each group
control_sim = np.random.binomial(1, p_control, 100000)
treatment_sim = np.random.binomial(1, p_treatment, n_sim)

# Vector of differences
diffs = treatment_sim - control_sim[:10000]

# Cumulative average of the differences
cum_avg = np.cumsum(diffs) / np.arange(1, len(diffs) + 1)

# Plot
plt.figure(figsize=(8, 4))
plt.plot(cum_avg, label='Cumulative Average Difference', color='orange')
plt.axhline(y=p_treatment - p_control, color='red', linestyle='--', label='True Difference (0.004)')
plt.title("Law of Large Numbers: Cumulative Average of Simulated Differences")
plt.xlabel("Simulation Iteration")
plt.ylabel("Cumulative Average Difference")
plt.legend()
plt.tight_layout()
plt.show()

#### Interpretation
This plot demonstrates the Law of Large Numbers. As we simulate more and more observations, the cumulative average of the differences converges toward the true mean difference (0.004). Initially, there’s randomness and fluctuation, but the line stabilizes as the number of observations increases.

This convergence is the foundation for using sample averages to estimate population parameters and underpins why large sample sizes give us more reliable estimates in experiments.


### Central Limit Theorem

_to do: Make 4 histograms like those on slide 44 from our first class at sample sizes 50, 200, 500, and 1000 and explain these plots to the reader. To do this for a sample size of e.g. 50, take 50 draws from each of the control and treatment distributions, and calculate the average difference between those draws. Then repeat that process 999 more times so that you have 1000 averages. Plot the histogram of those averages. Comment on whether zero is in the "middle" of the distribution or whether it's in the "tail."_


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p_control = 0.018
p_treatment = 0.022
true_diff = p_treatment - p_control
sample_sizes = [50, 200, 500, 1000]
n_reps = 1000

np.random.seed(42)
histograms = {}

# Simulate average differences for each sample size
for n in sample_sizes:
    diffs = []
    for _ in range(n_reps):
        control_sample = np.random.binomial(1, p_control, n)
        treatment_sample = np.random.binomial(1, p_treatment, n)
        diff = treatment_sample.mean() - control_sample.mean()
        diffs.append(diff)
    histograms[n] = diffs

# Plot histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for i, n in enumerate(sample_sizes):
    axes[i].hist(histograms[n], bins=30, color='skyblue', edgecolor='black')
    axes[i].axvline(0, color='red', linestyle='--', label="Zero Reference")
    axes[i].axvline(true_diff, color='green', linestyle='--', label="True Difference (0.004)")
    axes[i].set_title(f"Sample Size = {n}")
    axes[i].set_xlabel("Mean Difference")
    axes[i].set_ylabel("Frequency")
    axes[i].legend()

plt.tight_layout()
plt.show()

#### Interpretation
These four histograms illustrate how the sampling distribution of the difference in means behaves at increasing sample sizes:

At n = 50, the distribution is quite wide and skewed. Zero is within the center-ish but not tightly.

As sample size increases, the distribution becomes tighter, more symmetric, and centered.

By n = 1000, the distribution of average differences closely resembles a normal distribution centered near the true mean difference (0.004).

This is a direct illustration of the Central Limit Theorem:

As sample size increases, the distribution of the sample mean difference becomes approximately normal, regardless of the original distribution shape.

Also note: zero shifts from being more "middle-ish" in smaller samples to being closer to the tail as the signal (the true effect) dominates the noise.
